### Splitiing the data set to test and train

In [15]:
import pandas as pd
import random
train_frac = 0.8

rating_path = '../data/ml-latest-small/ratings.csv'
rating_pd = pd.read_csv(rating_path)
df_shuffled = rating_pd.sample(frac=1).reset_index(drop=True)
# df_shuffled.head()

train_set = df_shuffled.iloc[:int(train_frac * len(df_shuffled))]
train_set.to_csv(path_or_buf='../data/train_set.csv', index=False)
test_set = df_shuffled.iloc[int(train_frac * len(df_shuffled)):]
test_set.to_csv(path_or_buf='../data/test_set.csv', index=False)

In [16]:
train_path = '../data/train_set.csv'
len(pd.read_csv(train_path))

80668

In [14]:
int(0.8 * len(df_shuffled))

80668

In [1]:
import numpy as np
import pandas as pd

In [2]:
latent_dim = 100
sigma = 0.5
sigma_p = 0.5
sigma_q = 0.5
mu = 0
rating_path = '../data/ml-latest-small/ratings.csv'
train_path = '../data/train_set.csv'

In [17]:
rating_pd = pd.read_csv(train_path)
num_user = np.unique(rating_pd.values[:, 0]).shape[0]
num_movie = np.unique(rating_pd.values[:, 1]).shape[0]
rating_pd.head()

,userId,movieId,rating,timestamp
0,438,4344,3.5,1105664636
1,323,230,3.0,1422640640
2,226,520,4.0,1095663071
3,103,1230,4.0,1432086587
4,68,1961,2.5,1158531027


In [18]:
r_matrix = np.zeros((num_user, num_movie))
p_matrix = np.random.normal(mu, sigma_p, (num_user, latent_dim))
q_matrix = np.random.normal(mu, sigma_q, (num_movie, latent_dim))

movie_dic = {}
idx = 0
for movie_id in np.unique(rating_pd.values[:, 1]):
    movie_dic[movie_id] = idx
    idx += 1

for index, row in rating_pd.iterrows():
    r_matrix[int(row['userId'] - 1), movie_dic[row['movieId']]] = row['rating']

In [19]:
def MSE (r, p, q):
    p_qt_matrix = np.dot(p, q.T)
    result = r - p_qt_matrix
    I = np.zeros_like(r)
    I[r != 0] = 0.5
    result = np.power(result, 2)
    result = np.multiply(I, result)
    return np.sum(result)
    
def L2_loss(sig1, sig2, matrix):
    result = np.power(matrix, 2)
    result = np.sum(result)
    constant = sig1 / float(sig2 * 2)
    return constant * result    

In [20]:
error = MSE(r_matrix, p_matrix, q_matrix) + L2_loss(sigma, sigma_q, q_matrix) + L2_loss(sigma, sigma_p, p_matrix)

In [21]:
def p_q_derivative (r, p, q, sigma, sigma_p, sigma_q):
    p_qt_matrix = np.dot(p, q.T)
    residual = r - p_qt_matrix
    I = np.zeros_like(r)
    I[r != 0] = 1
    residual = np.multiply(I, residual)
    p_derivative = -1.0 * np.dot(residual, q) + sigma / sigma_p * p
    q_derivative = -1.0 * np.dot(residual.T, p) + sigma / sigma_q * q
    return p_derivative, q_derivative

In [25]:
learning_rate = 0.0001

for i in range(3001):
    p_derivative, q_derivative = p_q_derivative (r_matrix, p_matrix, q_matrix, sigma, sigma_p, sigma_q)
    p_matrix = p_matrix - learning_rate * p_derivative 
    q_matrix = q_matrix - learning_rate * q_derivative 
    error = MSE(r_matrix, p_matrix, q_matrix)
    if i % 50 == 0:
        print(error)

552.8159613069719
544.9576567123416
537.3322820166218
529.9328904284187
522.7527539772551
515.7853545054434
509.0243751563792
502.4636923350748
496.0973681171627
489.91964308298407
483.92492955387115
478.10780520827467
472.4630070559875
466.98542574936164
461.67010021109684
456.5122125588938
451.50708330802775
446.65016683365025
441.93704707542594
437.3634334679022
432.92515708083647
428.6181669544679
424.4385266156042
420.3824107611051
416.44610209622357
412.62598831596443
408.91855921845854
405.3204039400299
401.8282083023975
398.4387522631312
395.1489074611611
391.9556348497974
388.8559824102936
385.84708293964263
382.9261519067838
380.0904853719804
377.3374579645874
374.6645209149273
372.0692001364034
369.5490943544032
367.10187327892044
364.7252758181602
362.4171083307356
360.17524291434114
357.9976157290614
355.8822253537243
353.82713117392353
351.83045180053387
349.8903635177175
348.0050987595958
346.172944614867
344.3922413588035
342.66138101215887
340.97880592659254
339.343007

In [29]:
learning_rate = 0.001

for i in range(3001):
    p_derivative, q_derivative = p_q_derivative (r_matrix, p_matrix, q_matrix, sigma, sigma_p, sigma_q)
    p_matrix = p_matrix - learning_rate * p_derivative 
    q_matrix = q_matrix - learning_rate * q_derivative 
    error = MSE(r_matrix, p_matrix, q_matrix)
    if i % 50 == 0:
        print(error)

286.31201148898214
283.6232793111692
281.3648113794037
279.4396718525381
277.77456927167367
276.31349268253564
275.01341493212675
273.84117560955457
272.7711611135824
271.78355419876425
270.8630014862388
269.99759309253307
269.17807814205327
268.3972600230148
267.6495293696989
266.93050296891124
266.2367443414194
265.5655474208157
264.91476905345417
264.2826993241195
263.66796122309484
263.06943309597284
262.48618879661876
261.91745160172695
261.36255882264805
260.82093472802535
260.292069915881
259.7755056816889
259.2708222467195
258.7776299590554
258.29556277394124
257.8242734724392
257.3634301970237
256.9127139767859
256.471816988851
256.0404413607462
255.61829836403317
255.20510788530373
254.8005980885315
254.40450520455406
254.01657340031386
253.63655469349794
253.26420888820638
252.89930351483522
252.54161376309148
252.190922401296
251.8470196782439
251.50970320614059
251.17877782468048
250.85405544741548
250.53535489221707
250.22250169802953
249.91532793029836
249.61367197748297

check the number of movie and user in the train set to be equal to whole 